In [2]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import matplotlib.pyplot as plt
import math
import librosa
import pandas as pd

import plotly.graph_objs as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score,balanced_accuracy_score,f1_score
from sklearn.model_selection import cross_validate
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

names=['drummer', 'session', 'id', 'style', 'bpm', 'beat_type',
       'time_signature', 'midi_filename', 'audio_filename', 'duration',
       'split', 'audio', 'mfcc_mean', 'mfcc_std', 'delta_mfcc_mean',
       'delta_mfcc_std', 'centroid_mean', 'centroid_std', 'rms_mean',
       'rms_std', 'zcr_mean', 'zcr_std', 'crest_mean', 'crest_std',
       'flux_mean', 'flux_std', 'onsets_mean', 'onsets_std', 'mfcc1_mean',
       'mfcc2_mean', 'mfcc3_mean', 'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean',
       'mfcc7_mean', 'mfcc8_mean', 'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean',
       'mfcc12_mean', 'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std',
       'mfcc4_std', 'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std',
       'mfcc9_std', 'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std', 'onsets_low_mean',
       'onsets_low_std','onsets_mid_mean', 'onsets_mid_std',
       'onsets_high_mean', 'onsets_high_std']
df=pd.DataFrame(np.load('./resources/working_data/data.npy',allow_pickle=True),columns=names)

In [3]:
X = df[['mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std','rms_mean','rms_std','zcr_mean','zcr_std','crest_mean','crest_std','flux_mean',
       'flux_std','onsets_mean', 'onsets_std','onsets_mid_mean', 'onsets_mid_std',
       'onsets_high_mean', 'onsets_high_std',
       'bpm']]
y=df[['style']]

# for col in X.columns:
#     #df[col] = df[col]/max(np.abs(df[col]))
#     mean = np.mean(X[col])
#     std = np.std(X[col])
#     X[col] = (X[col] - mean)/std # z-score normalization

X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.4)

X_train = X_train.to_numpy()
X_test1 = X_test1.to_numpy()
y_train = y_train.to_numpy()
y_test1 = y_test1.to_numpy()

X_test,X_valid,y_test,y_valid = train_test_split(X_test1,y_test1,test_size=0.5)

# X_train = X_train.reshape(-1, 69)
# X_test = X_test.reshape(-1, 69)
# X_valid = X_valid.reshape(-1, 69)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

In [4]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

In [5]:
le = LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)
y_valid=le.transform(y_valid)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [6]:
path='/Users/vedant/Desktop/Programming/ACA-project/trained_models'

In [7]:
X_train[0].shape

(69,)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(148, 69)
(49, 69)
(50, 69)


In [9]:
import pandas as pd
import tensorflow as tf
def train_models(X_train,y_train_hot,X_test,y_test_hot,epochs,batch_size,lr,layer1_nodes,optimiser,loss,verbose,save=False,seed_value=42):
    tf.random.set_seed(seed_value)
    model = Sequential()
    input_shape = (69,1)#(128, 87, 1)
    # model.add(Dense(128, activation='relu'))
    # model.add(Dense(6, activation='softmax'))
    model.add(Dense(layer1_nodes,input_dim=69, activation='relu'))
    model.add(Dense(layer1_nodes,input_dim=32, activation='relu'))
    model.add(Dense(layer1_nodes,input_dim=8, activation='relu'))
    # model.add(Dense(32, activation='relu'))
    # model.add(Dense(16, activation='relu'))
    # model.add(Dense(10, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    if optimiser=='adadelta':
        optim=keras.optimizers.Adadelta(learning_rate=lr)
    if optimiser == 'adam':
        optim=keras.optimizers.Adam(learning_rate=lr)

    if loss == 'crossentropy':
        loss_fn = keras.losses.categorical_crossentropy

    model.compile(loss=loss_fn,
                optimizer=optim,
                metrics=['accuracy'])
    model.build(input_shape)
    model.summary()
    # fit the model
    history=model.fit(X_train, y_train_hot,
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose,
            validation_data=(X_test, y_test_hot))
    training_loss=history.history['loss']
    validation_loss=history.history['val_loss']
    training_acc=history.history['accuracy']
    validation_acc=history.history['val_accuracy']
    df=pd.DataFrame()
    df['training_loss'] = training_loss
    df['validation_loss'] = validation_loss
    df['training_acc'] = training_acc
    df['validation_acc'] = validation_acc
    lr_str=str(lr).replace('.','_')
    model_name=f'fcnn_layer1-{layer1_nodes}_batch-{batch_size}_epochs-{epochs}_lr-{lr_str}'
    if save:
        model.save(f'{path}/{model_name}')
        df.to_csv(f'{path}/{model_name}.csv')

    trace1 = go.Scatter(
    x=df.index,
        y=df['training_loss'],
        name='Training Loss'
    )
    trace2 = go.Scatter(
        x=df.index,
        y=df['validation_loss'],
        name='Validation Loss'
    )
    trace3 = go.Scatter(
        x=df.index,
        y=df['training_acc'],
        name='Training Accuracy',
        yaxis='y2'
    )
    trace4 = go.Scatter(
        x=df.index,
        y=df['validation_acc'],
        name='Validation Accuracy',
        yaxis='y2'
    )

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace3,secondary_y=True)
    fig.add_trace(trace4,secondary_y=True)
    fig.show()
    return model,df

In [11]:
model,df=train_models(X_train,y_train_hot,X_test,y_test_hot,150,128,0.0005,32,'adam','crossentropy',verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                2240      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 132       
Total params: 4,484
Trainable params: 4,484
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
2/2 [==============================] - 1s 126ms/step - loss: 1.3062 - accuracy: 0.4527 - val_loss: 1.2412 - val_accuracy: 0.4694
Epoch 2/150
2/2 [==============================] - 0s 15ms/step - loss: 1.2733 - accuracy: 0.5068 - val_loss:

In [24]:
y_pred=model.predict(X_valid)
y_pred[np.where(y_pred==np.max(y_pred))] = 1
y_pred=np.argmax(y_pred,axis=1)

In [25]:
y_pred.shape

(50,)

In [26]:
score=balanced_accuracy_score(y_valid, y_pred)
cm = confusion_matrix(y_valid, y_pred)
f1=f1_score(y_valid,y_pred,average='macro')
acc = accuracy_score(y_valid, y_pred)

print(cm)
print("Accuracy Score: ",acc)
print("Macro accuracy Score:",score)
print(f"F1 score: {f1}")

[[ 1  0  0  3]
 [ 1  3  0  2]
 [ 0  0  2  3]
 [ 1  3  0 31]]
Accuracy Score:  0.74
Macro accuracy Score: 0.5089285714285714
F1 score: 0.5487451737451737
